# Workflow for processing DAS data from soil desiccation at 
# BC cribs and trenches on the 200 West Hanford Site

Read in Hdf5 files

In [ ]:
S = read_data("Hdf5","filename")

# Merge and ungap

In [ ]:
merge!(S)
ungap!(S)

## Downsample data 
- Removes mean from each channel in S.
- Detrends each channel in S.
- Downsamples data to sampling rate fs
- Phase-shifts data to begin at 00:00:00.0
- This is important for data that does not begin exactly on the sampling rate, e.g. the starttime is 2006-02-01T00:00:00.020.

In [ ]:
fs = 20.
process_raw!(S,fs)

Note that now S has sampling rate fs = 20.0, 
has half as many points (nx = XXXX) as before and the starttime has changed to XXXX.
Detrending and Demeaning
The demean and detrend functions are applied column wise.

# Detrend and Demean

In [ ]:
# may need to reshape
A = reshape(collect(1.:10.),5,2) # this would reshape to a 5,2

In [ ]:
demean(A)
detrend(A)

demean and detrend also work on RawData and CorrData structure. If R is a RawData, 
then the windowed data stored in R.x can be detrended in-place using

In [ ]:
detrend!(R)
# or allocated to a new RawData using
Rd = detrend(R)

# Amplitude Normalization
Time-domain normalization is used to suppress high-amplitude signals, such as earthquakes or instrumental irregularities. 
SeisNoise provides time-normalization functions for:
- one-bit normalization: onebit
- clipping: clip
- running mean normalization: smooth
- suppressing high-amplitude signals: mute

# Filtering a RawData or CorrData

julia> freqmin, freqmax = 1., 10. # low and high frequencies in Hz
julia> ncorners = 4 # number of corners in Butterworth filter
julia> zerophase = true # if true, apply filter twice for no phase shifting
julia> S = get_data("IRIS","TA.V04C..BHZ",s="2006-02-01T00:00:00",t="2006-02-01T01:00:00")
julia> SeisIO.demean!(S) # remove mean
julia> SeisIO.detrend!(S) # remove linear trend
julia> SeisIO.taper!(S) # taper - defaults to 5% taper on either side of the trace
julia> bandpass(S,freqmin,freqmax,corners=corners,zerophase=zerophase)

# Filtering
SeisNoise.jl provides bandpass, bandstop, lowpass and highpass filters built from DSP.jl
Due to multiple dispatch in Julia, filter functions in SeisNoise.jl work on either the data in RawData or CorrData objects or 
directly with Julia Arrays. SeisNoise.jl uses Butterworth filters with a default of 4 corners.